# The prime network

#### Riddler Classic for Friday May 6, 2022. Solution by [Laurent Lessard](https://laurentlessard.com)

This week's [Riddler Classic](https://fivethirtyeight.com/features/can-you-build-the-longest-ladder/) is about a network of prime numbers. Here is the problem:

> As you may know, a word ladder is a sequence of words in which exactly one letter changes from one word to the next. And an optimal word ladder is a word ladder that gets you from the initial word to the final word using as few other words as possible. For example, to go from DOG to CAT, an optimal word ladder has just four total words (including DOG and CAT): DOG, DOT, COT and CAT.
>
> For this puzzle, Michael similarly defines optimal prime ladders. Every number in the ladder must be prime, with exactly one digit changing from one number to the next. For example, here is an optimal prime ladder that goes from 97 to 53 with just four total numbers: 97, 17, 13 and 53.
>
> This is tied for the longest optimal prime ladder consisting of two-digit primes. What is the longest optimal prime ladder for three-digit primes? Four-digit primes? Five-digit primes?
>
> Extra credit: What is the longest optimal prime ladder for six-digit primes?

## Solution

Imagine a graph, where each node is a different $N$-digit prime. We draw an edge connecting any two nodes if the corresponding primes differ by exactly one digit. Then, in graph theory terms,
- A "ladder" from $p_1$ to $p_2$ is a [path](https://en.wikipedia.org/wiki/Path_(graph_theory)) from node $p_1$ to node $p_2$.
- An "optimal ladder" from $p_1$ to $p_2$ is the path of minimum [distance](https://en.wikipedia.org/wiki/Distance_(graph_theory)) between $p_1$ and $p_2$.
- The "longest optimal ladder" is the [diameter](https://en.wikipedia.org/wiki/Distance_(graph_theory)#:~:text=To%20find%20the%20diameter%20of,the%20diameter%20of%20the%20graph.) of the graph.

There exist [polynomial-time algorithms](https://stackoverflow.com/questions/15646307/algorithm-for-diameter-of-graph) for finding the diameter of a graph, which is what we implemented here.

Here are the solutions I obtained. For each, I will give the length of the longest optimal ladder, which is the same as the diameter of the graph plus one (since the diameter counts the number of edges, not the number of nodes). I also provide one possible example of a longest optimal ladder.
- $N=2$. Length 4. Example: `[23, 73, 71, 31]`
- $N=3$. Length 7. Example: `[389, 379, 179, 109, 101, 701, 761]`
- $N=4$. Length 9. Example: `[2441, 5441, 5449, 5849, 5869, 6869, 6899, 6199, 9199]`
- $N=5$. Length 11. Example: `[88259, 78259, 76259, 96259, 96269, 96469, 96461, 96661, 99661, 99761, 99721]`
- $N=6$. Length 13. Example: `[440497, 410497, 410491, 710491, 710441, 710443, 717443, 917443, 917843, 907843, 905843, 905833, 995833]`

For $N\leq 5$, the graph is [connected](https://en.wikipedia.org/wiki/Connectivity_(graph_theory)). But when $N=6$, interestingly, the graph is disconnected! The graph for $N=6$ has 68,906 nodes, but 7 connected components! They are: `[294001]`, `[505447]`, `[584141]`, `[604171]`, `[929573]`, `[971767]`, and then everything else. This means that if you start with any of these six primes, there is no way to change a single digit and obtain another prime. My code is below.

# My Code

In [384]:
from pyvis.network import Network
from itertools import combinations

## Make visualization

In [386]:
DIGITS = 3
plist = [n for n in primes(10**(DIGITS)-1) if n >= 10**(DIGITS-1)]
net = Network('600px','900px')
net.add_nodes(plist, size=[8]*len(plist))

for (p1,p2) in combinations(plist,2):
    if isnear(p1,p2):
        net.add_edge(p1,p2)

# get adjacency matrix
graph = net.get_adj_list()
        
# MAKE VISUALIZATION
net.toggle_physics(status=True)
net.prep_notebook()
net.show('netplot.html')

---
## Use csr representation

In [387]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path, connected_components
from itertools import combinations
import numpy as np

In [388]:
def primes(n):
    """ Returns  a list of primes < n """
    sieve = [True] * n
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i]:
            sieve[i*i::2*i]=[False]*((n-i*i-1)//(2*i)+1)
    return [2] + [i for i in range(3,n,2) if sieve[i]]

In [389]:
def isnear(p1,p2):
    """ Returns TRUE if p1 and p2 differ by one digit """
    string1 = str(p1)
    string2 = str(p2)
    count_diffs = 0
    for a, b in zip(string1, string2):
        if a!=b:
            if count_diffs: return False
            count_diffs += 1
    return count_diffs == 1

In [390]:
def isnear_str(string1,string2):
    """ Returns TRUE if p1 and p2 differ by one digit """
    count_diffs = 0
    for a, b in zip(string1, string2):
        if a!=b:
            if count_diffs: return False
            count_diffs += 1
    return count_diffs == 1

In [391]:
def get_path(plist,Pr, i, j):
    path = [int(plist[j])]
    k = j
    while Pr[i, k] != -9999:
        path.append(int(plist[Pr[i, k]]))
        k = Pr[i, k]
    return path[::-1]

In [392]:
%%time

DIGITS = 3

plist = [str(n) for n in primes(10**(DIGITS)-1) if n >= 10**(DIGITS-1)]

data = []
row_ind = []
col_ind = []

for (i,j) in combinations(range(len(plist)),2):
    if isnear_str(plist[i],plist[j]):
        data.append(1)
        row_ind.append(i)
        col_ind.append(j)

# build matrix sparse adjacency matrix
mat = csr_matrix((data,(row_ind,col_ind)),(len(plist),len(plist)))

Wall time: 19.2 ms


In [393]:
%%time

dist_matrix, pred = shortest_path(csgraph=mat, directed=False, return_predecessors=True)

# deal with the possibility that the graph has multiple connected components
if connected_components(mat, directed=False, return_labels=False) > 1:
    print('There are multiple connected components!')
    dist_matrix = np.where(dist_matrix == np.inf, -9999, dist_matrix)

# find longest path (graph diameter)
(i1,i2) = np.unravel_index(dist_matrix.argmax(), dist_matrix.shape)
pp = get_path(plist,pred,i1,i2)
print(len(pp)-1)
print(pp)

6
[389, 379, 179, 109, 101, 701, 761]
Wall time: 4.99 ms
